In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils

In [2]:
from keras.models import Model, load_model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Flatten
from keras.models import Model

Using TensorFlow backend.


In [3]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [5]:
word2ind, ind2word = utils.load_index_word_map()

In [6]:
vocab_size = len(word2ind)
n_tokens = 100

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, n_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, n_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(n_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [8]:
# Compile & run training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
encoder_input_data = pd.read_csv('encoder_input.csv').values
decoder_input_data = pd.read_csv('decoder_input.csv').values
decoder_target_data = pd.read_csv('decoder_target.csv').values
decoder_target_data = decoder_target_data.reshape((decoder_target_data.shape[0], decoder_target_data.shape[1], -1))

In [10]:
def train_valid_split(length, test_size=0.3, random_state=43):
    np.random.RandomState(seed=random_state)
    choices = list(range(length))
    val_choices = np.random.choice(
        choices,
        int(length*test_size),
        replace=False
    ).tolist()
    train_choices = list(set(choices) - set(val_choices))
    return train_choices, val_choices

In [11]:
encoder_input_data.shape

(397605, 20)

In [12]:
decoder_input_data.shape

(397605, 21)

In [13]:
decoder_target_data.shape

(397605, 21, 1)

In [14]:
train_set, val_set = train_valid_split(encoder_input_data.shape[0])

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, None, 50)      1645850     input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, None, 100), ( 60400       embedding_1[0][0]       

In [17]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit(
    [encoder_input_data[train_set], decoder_input_data[train_set]],
    decoder_target_data[train_set],
    validation_data=([encoder_input_data[val_set], decoder_input_data[val_set]], decoder_target_data[val_set]),
    batch_size=500,
    epochs=5
)

Train on 278324 samples, validate on 119281 samples
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[500,21,32917]
	 [[Node: loss/dense_1_loss/Log = Log[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/dense_1_loss/clip_by_value)]]
	 [[Node: metrics/acc/Mean/_63 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3787_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'loss/dense_1_loss/Log', defined at:
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/f

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [20]:
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')